<a href="https://colab.research.google.com/github/maheravi/Deep-Learning/blob/main/FAMnist_PyTorch/FAMnist_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import wandb

In [99]:
wandb.init(project="FAMNist", entity="ma_heravi")

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST

In [100]:
# hyperparameters
latent_size = 10
disc_inp_sz = 28*28
img_size = 28
epochs = 10
batch_size = 32
lr = 0.001
wandb.config = {
  "learning_rate": lr,
  "epochs": epochs,
  "batch_size": 32
}

In [101]:
class MyModel(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(MyModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dims, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.25),
            
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.5),
            
            nn.Linear(64, output_dims),
            nn.Sigmoid(),
                        )
        
    def forward(self, input_t):
      input_t = input_t.reshape((input_t.shape[0], 784))
      return self.fc(input_t)

In [89]:
 device = torch.device("cuda")
# device = torch.device("cpu")

model = MyModel(disc_inp_sz, latent_size).to(device)

model = model.to(device)
model.train(True)

MyModel(
  (fc): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Dropout(p=0.5, inplace=False)
    (12): Linear(in_features=64, out_features=10, bias=True)
    (13): Sigmoid()
  )
)

In [90]:
def calc_acc(preds, labels):
    _, preds_max = torch.max(preds, 1)
    acc = torch.sum(preds_max == labels.data, dtype=torch.float64) / len(preds)
    return acc

In [91]:
# Data Preparing

transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize((0), (1))
])

dataset = torchvision.datasets.FashionMNIST("./dataset", train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [92]:
 # compile
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_function = torch.nn.CrossEntropyLoss()

In [103]:
# train

for epoch in range(1, epochs+1):
    train_loss = 0.0
    train_acc = 0.0
    for images, labels in train_data_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        # 1- forwarding
        preds = model(images)
        # 2- backwarding 
        loss = loss_function(preds, labels)
        loss.backward()
        # 3- Update
        optimizer.step()

        train_loss += loss
        train_acc += calc_acc(preds, labels)
    
    total_loss = train_loss / len(train_data_loader)
    total_acc = train_acc / len(train_data_loader)

    print(f"Epoch: {epoch}, Loss: {total_loss}, Acc: {total_acc}")
    wandb.log({'epochs':  epoch + 1,
              'loss': total_loss,
              'acc': total_acc
                              })

Epoch: 1, Loss: 1.5897778272628784, Acc: 0.8096666666666668
Epoch: 2, Loss: 1.5875996351242065, Acc: 0.8134666666666667
Epoch: 3, Loss: 1.5891987085342407, Acc: 0.8096166666666667
Epoch: 4, Loss: 1.587140440940857, Acc: 0.8137500000000001
Epoch: 5, Loss: 1.5859978199005127, Acc: 0.8131
Epoch: 6, Loss: 1.5859174728393555, Acc: 0.8155333333333333
Epoch: 7, Loss: 1.5866419076919556, Acc: 0.8155333333333333
Epoch: 8, Loss: 1.586602807044983, Acc: 0.8125833333333333
Epoch: 9, Loss: 1.5848454236984253, Acc: 0.8156166666666667
Epoch: 10, Loss: 1.5847679376602173, Acc: 0.8157000000000001


In [104]:
# save
torch.save(model.state_dict(), "FAMnist.pth")

In [ ]:
# inference

import cv2
import numpy as np


# model.train(False)
model.eval()

# preprocess
img = cv2.imread("3.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
tensor = transform(img).unsqueeze(0).to(device)

# process
preds = model(tensor)

# postprocess
preds = preds.cpu().detach().numpy()
output = np.argmax(preds)
output

3